# Dual Simplex examlpe
### problem as follows:
$$
\max \ \ x_1 + 3x_2 \\
\begin{align*}
\\
\  x_1 & + x_2 & \leq 8\\
\ -2x_1 & + 7x_2 & \leq 20\\
\  2x_1 & -3x_2   &\leq -5\\
\end{align*}
$$

In [9]:
import numpy as np
import util as u
from fractions import Fraction

SLACK_COUNT = 3

A = np.array([[1, -2, 2],
              [1,  7, -3]], dtype=object)
C = np.array([1, 3], dtype=object)
B = np.array([8, 20, -5], dtype=object)

# Convert all entries to Fractions
A = np.vectorize(Fraction)(A)
C = np.vectorize(Fraction)(C)
B = np.vectorize(Fraction)(B)

# Build identity and zeros explicitly as Fractions
I = np.array([[Fraction(int(i == j)) for j in range(SLACK_COUNT)] for i in range(SLACK_COUNT)], dtype=object)
Z = np.array([Fraction(0) for _ in range(SLACK_COUNT)], dtype=object)

# Build the tableau
T = np.concatenate([A.T, I], axis=1)
T = np.column_stack((T, Z))
T = np.column_stack((T, B))
T = np.vstack((T, np.concatenate([C, np.full(SLACK_COUNT, Fraction(0), dtype=object), [Fraction(1), Fraction(0)]])))

u.tableau(T)


|-------+-------+-------+-------+-------+-------+-------+
|    x1 |    x2 |    x3 |    x4 |    x5 |    -z |     b |
|-------+-------+-------+-------+-------+-------+-------+
|     1 |     1 |     1 |     0 |     0 |     0 |     8 |
|    -2 |     7 |     0 |     1 |     0 |     0 |    20 |
|     2 |    -3 |     0 |     0 |     1 |     0 |    -5 |
|-------+-------+-------+-------+-------+-------+-------+
|     1 |     3 |     0 |     0 |     0 |     1 |     0 |
|-------+-------+-------+-------+-------+-------+-------+


In [10]:
# Is infeasible, perform Dual Simplex:

# 2/1 = 2, -3/3 = -1
# x2 enters, s3 leaves

# III * -(1/3)
T[2] = T[2] * Fraction(-1, 3)

# I - III
T[0] = T[0] - T[2]

# II - 7 * III
T[1] = T[1] - T[2] * Fraction(7)

# IV - 2 * III
T[3] = T[3] - T[2] * Fraction(3)

# 1st iteration complete
u.tableau(T)

|-------+-------+-------+-------+-------+-------+-------+
|    x1 |    x2 |    x3 |    x4 |    x5 |    -z |     b |
|-------+-------+-------+-------+-------+-------+-------+
|   5/3 |     0 |     1 |     0 |   1/3 |     0 |  19/3 |
|   8/3 |     0 |     0 |     1 |   7/3 |     0 |  25/3 |
|  -2/3 |     1 |     0 |     0 |  -1/3 |     0 |   5/3 |
|-------+-------+-------+-------+-------+-------+-------+
|     3 |     0 |     0 |     0 |     1 |     1 |    -5 |
|-------+-------+-------+-------+-------+-------+-------+


In [11]:
# continue with normal Simplex from here

# x1 enters, s1 leaves

# I * 3/5
T[0] = T[0] * Fraction(3, 5)

# II - I * 8/3
T[1] = T[1] - T[0] * Fraction(8, 3)

# III + I * 2/3
T[2] = T[2] + T[0] * Fraction(2, 3)

# IV - 3 * I
T[3] = T[3] - T[0] * Fraction(3)

# 2nd iteration complete
u.tableau(T)

|-------+-------+-------+-------+-------+-------+-------+
|    x1 |    x2 |    x3 |    x4 |    x5 |    -z |     b |
|-------+-------+-------+-------+-------+-------+-------+
|     1 |     0 |   3/5 |     0 |   1/5 |     0 |  19/5 |
|     0 |     0 |  -8/5 |     1 |   9/5 |     0 |  -9/5 |
|     0 |     1 |   2/5 |     0 |  -1/5 |     0 |  21/5 |
|-------+-------+-------+-------+-------+-------+-------+
|     0 |     0 |  -9/5 |     0 |   2/5 |     1 | -82/5 |
|-------+-------+-------+-------+-------+-------+-------+


In [12]:
# again infeasible, perform Dual Simplex:

# s1 enters, s2 leaves

# II * -5/8
T[1] = T[1] * Fraction(-5, 8)

# I - II * 3/5
T[0] = T[0] - T[1] * Fraction(3, 5)

# III - II * 2/5
T[2] = T[2] - T[1] * Fraction(2, 5)

# IV + II * 9/5
T[3] = T[3] + T[1] * Fraction(9, 5)

# 3rd iteration complete
u.tableau(T)

|-------+-------+-------+-------+-------+-------+-------+
|    x1 |    x2 |    x3 |    x4 |    x5 |    -z |     b |
|-------+-------+-------+-------+-------+-------+-------+
|     1 |     0 |     0 |   3/8 |   7/8 |     0 |  25/8 |
|     0 |     0 |     1 |  -5/8 |  -9/8 |     0 |   9/8 |
|     0 |     1 |     0 |   1/4 |   1/4 |     0 |  15/4 |
|-------+-------+-------+-------+-------+-------+-------+
|     0 |     0 |     0 |  -9/8 | -13/8 |     1 | -115/8 |
|-------+-------+-------+-------+-------+-------+-------+


$$
\textbf{In Simplex:}
$$

$$
j^* = \arg\max_j \left( c_j \cdot \theta_j \right),
\quad \text{where} \quad 
\theta_j = \min_i \left\{ \frac{b_i}{a_{ij}} \; : \; a_{ij} > 0 \right\}
$$

$$
\text{Pivot on } (i^*, j^*) \text{ where } 
i^* = \min{i} \text{ from chosen } \theta_j
$$

---

$$
\textbf{In Dual Simplex:}
$$

$$
i^* = \arg\min_i \{ b_i \}
\quad \text{(row with most negative } b_i)
$$

$$
j^* = \arg\min_j \left\{ \frac{c_j - z_j}{a_{i^*j}} \; : \; a_{i^*j} < 0 \right\}
$$

$$
\text{Pivot on } (i^*, j^*)
$$
